In [1]:
import sys
sys.path.append('../../preprocess_assets/')

In [2]:
import glob
import os
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier, LogisticRegression
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, LSTM, Embedding
import tensorflow as tf
from features_extraction import *
from data_shuffling_split import *
from ara_vec_preprocess_configs import *
from ml_modeling import *
from keras_models import *

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
strat_train_set = read_file("train/strat_train_set.csv")
strat_train_set.head()

,label,text
0,1,مع انتشار الامراض ومنها الكورونا اليكم النصيحه...
1,0,1 دراسه بحثيه صادره معهد الدراسات الاستراتيجي...
2,1,بدات في مدينه وهان في الصين انتقلت لبعض المقاط...
3,1,coronarvirus CoronaOutbreak فيروس_كورونا
4,1,الاداره_العامه_للطيران_المدني اجراءات وقاءيه ح...


In [4]:
x_train_text, x_val_text, y_train, y_val = prepare_data(strat_train_set)

The number of instances in the training data after StratifiedShuffleSplit are:  8643
The number of instances in the testing data after StratifiedShuffleSplit are:   177
The number of trainin instances:  8643
The number of validation instances:  177
The number of trainin labels :  8643
The number of validation labels :  177


In [5]:
x_train_text_tokenized = tokenize_using_nltk_TreebankWordTokenizer(x_train_text)

print("Before Tokenization : \n", x_train_text[:3])
print("="*50)
print("After Tokenization : \n", x_train_text_tokenized[:3])
print("="*50)

x_val_text_tokenized = tokenize_using_nltk_TreebankWordTokenizer(x_val_text)

print("Before Tokenization : \n", x_val_text[:3])
print("="*50)
print("After Tokenization : \n", x_val_text_tokenized[:3])

fullgram_x_train_text_tokenized = get_all_ngrams(x_train_text_tokenized)
print("full gram tokenization : \n", fullgram_x_train_text_tokenized[:3])
print("="*50)
fullgram_x_val_text_tokenized = get_all_ngrams(x_val_text_tokenized)
print("full gram tokenization : \n", fullgram_x_val_text_tokenized[:3])
print("="*50)

Before Tokenization : 
 ['حراميه البرلمان الفاسدين يريدون يدمرون العراق لسبعين سنه قادمه باتفاقيه الصين ام الصناعه الفاشله تشتري صناعه صينيه ينزل سعرها للنص لانها فاشله لعنه الله علي ايران وعلي الحكومه العراقيه الفاسده والمعممين الفاسدين والميليشيات الخونه والاحزاب الاسلاميه المدسوسه عميله ايران', 'الصين سوف تسيطر علي العالم سياسيا وعسكريا قريبا', 'لا للتبادل التجاري مع الصين ما دامت تكذب وتخادع وتسرق ']
After Tokenization : 
 [['حراميه', 'البرلمان', 'الفاسدين', 'يريدون', 'يدمرون', 'العراق', 'لسبعين', 'سنه', 'قادمه', 'باتفاقيه', 'الصين', 'ام', 'الصناعه', 'الفاشله', 'تشتري', 'صناعه', 'صينيه', 'ينزل', 'سعرها', 'للنص', 'لانها', 'فاشله', 'لعنه', 'الله', 'علي', 'ايران', 'وعلي', 'الحكومه', 'العراقيه', 'الفاسده', 'والمعممين', 'الفاسدين', 'والميليشيات', 'الخونه', 'والاحزاب', 'الاسلاميه', 'المدسوسه', 'عميله', 'ايران'], ['الصين', 'سوف', 'تسيطر', 'علي', 'العالم', 'سياسيا', 'وعسكريا', 'قريبا'], ['لا', 'للتبادل', 'التجاري', 'مع', 'الصين', 'ما', 'دامت', 'تكذب', 'وتخادع', 'وتسرق']]
Before Tokenizatio

# Our CBOW Word2Vec Model

In [6]:
our_word2vec_model = load_word2vec_model("../word2vec_models/rezk/skipgram_NS/sk_gr_negative_sampeling_fullgram_vec_size_300-d_min_count_100")

In [7]:
number_of_features = 300
max_len_str = 237
word2vec_path = "rezk/cbow/"
model_path_to_save = "../ml_models_saved/"
hid_num_neurons = 25
learning_rate = .00005
epochs = 10
estimators = voting_models()

performance_lr = keras.callbacks.ReduceLROnPlateau(factor=.5, patience=5)
RMSprop_optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate, rho=.9)


X_train_embed_matrix = text_to_matrix_using_word2vec(our_word2vec_model, fullgram_x_train_text_tokenized, max_len_str)
X_val_embed_matrix = text_to_matrix_using_word2vec(our_word2vec_model, fullgram_x_val_text_tokenized, max_len_str)
# Reshape because of deep learning model
X_train_embed_matrix = X_train_embed_matrix.reshape(X_train_embed_matrix.shape[0], max_len_str, number_of_features)
X_val_embed_matrix = X_val_embed_matrix.reshape(X_val_embed_matrix.shape[0], max_len_str, number_of_features)

(8643, 237, 300)
(8643, 71100)
[ 0.2402    -0.1918     0.1027    -0.0613     0.1823    -0.0956
  0.1512     0.249      0.1882    -0.0598    -0.00992   -0.2202
  0.0854     0.08923   -0.173      0.246      0.1532     0.07855
 -0.1445    -0.504      0.03442   -0.3127    -0.002003  -0.0835
 -0.1401    -0.3276    -0.6074    -0.3103    -0.2012    -0.005264
  0.06476   -0.1892    -0.07336    0.0315    -0.3127    -0.2084
 -0.0134    -0.3484     0.1562     0.2559    -0.02588   -0.0537
 -0.0007763 -0.0876     0.13      -0.1254    -0.2491     0.524
  0.1714     0.00745  ]
(177, 237, 300)
(177, 71100)
[ 0.0918   -0.011734  0.1099   -0.03403  -0.2878   -0.1137    0.1555
  0.5137   -0.04056   0.00814  -0.006405  0.127    -0.1863   -0.2913
 -0.1727   -0.00464  -0.2751   -0.2072   -0.1681   -0.5215    0.11334
  0.003372 -0.366     0.02788   0.3557   -0.1781   -0.1787   -0.2018
 -0.1284   -0.0605    0.478    -0.1995   -0.1384   -0.1465   -0.01225
  0.118     0.1432    0.2211    0.3       0.274    -0.1

# Our CBOW Word2Vec Model

# With  Rmsprob and  Batch Normalization

In [8]:
callbacks_ = keras_callbacks(word2vec_type="rezk_skipgram_NS_word2vec", model_type="Rmsprob_lstm_with_batch", learning_rate=learning_rate)
callbacks_.append(performance_lr)
model = lstm_with_batch_model_create(hid_num_neurons, max_len_str, number_of_features, dropout=.2)
model = seqential_model_compile(model, RMSprop_optimizer)
model.summary()

./../ml_models_saved/dl_models/tensor_logs/run_2022_05_09_23_58_39_rezk_skipgram_NS_word2vec_Rmsprob_lstm_with_batch_learning_rate=5e-05_


2022-05-09 23:58:39.997776: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-09 23:58:40.009086: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-09 23:58:40.010909: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-09 23:58:40.448301: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-09 23:58:40.450121: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from S

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 batch_normalization (BatchN  (None, 237, 300)         1200      
 ormalization)                                                   
                                                                 
 lstm (LSTM)                 (None, 237, 25)           32600     
                                                                 
 batch_normalization_1 (Batc  (None, 237, 25)          100       
 hNormalization)                                                 
                                                                 
 dropout (Dropout)           (None, 237, 25)           0         
                                                                 
 flatten (Flatten)           (None, 5925)              0         
                                                                 
 dense (Dense)               (None, 18)                1

In [9]:
history = model.fit(X_train_embed_matrix, y_train, batch_size=32, epochs=epochs, validation_data=(X_val_embed_matrix, y_val),
                   callbacks=callbacks_)

Epoch 1/10


2022-05-09 23:58:57.138094: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100


271/271 [==============================] - 16s 40ms/step - loss: 1.0044 - accuracy: 0.5588 - val_loss: 5.3490 - val_accuracy: 0.0000e+00 - lr: 5.0000e-05
Epoch 2/10
271/271 [==============================] - 10s 39ms/step - loss: 0.5751 - accuracy: 0.7179 - val_loss: 0.4173 - val_accuracy: 0.8249 - lr: 5.0000e-05
Epoch 3/10
271/271 [==============================] - 10s 37ms/step - loss: 0.4230 - accuracy: 0.8086 - val_loss: 0.3143 - val_accuracy: 0.8531 - lr: 5.0000e-05
Epoch 4/10
271/271 [==============================] - 10s 37ms/step - loss: 0.3603 - accuracy: 0.8444 - val_loss: 0.2678 - val_accuracy: 0.8870 - lr: 5.0000e-05
Epoch 5/10
271/271 [==============================] - 10s 38ms/step - loss: 0.3111 - accuracy: 0.8697 - val_loss: 0.2479 - val_accuracy: 0.8814 - lr: 5.0000e-05
Epoch 6/10
271/271 [==============================] - 10s 37ms/step - loss: 0.2876 - accuracy: 0.8823 - val_loss: 0.2392 - val_accuracy: 0.8927 - lr: 5.0000e-05
Epoch 7/10
271/271 [=====================

# Load best model & predict test set

In [10]:
strat_test_set = read_file("test/strat_test_set.csv")
strat_test_set.head()

,label,text
0,1,حساء خفافيش الصين ابداع القرف يعني
1,1,اجراءات هكذا استعدت مصر لمواجهه فيروس كورونا م...
2,1,ايش الفايده لابس كمامات ومو لابس قفازات
3,0,الجدير بالذكر ان في الصين بيستخدموا ال في كل ح...
4,0,ولو ترسل صواريخ الصين وقفنا دون برج الفيصليه


In [11]:
X_test_text = list(strat_test_set['text'])
y_test = strat_test_set['label'].values

In [12]:
X_test_text_tokenized = tokenize_using_nltk_TreebankWordTokenizer(X_test_text)

print("Before Tokenization : \n", X_test_text[:3])
print("="*50)
print("After Tokenization : \n", X_test_text_tokenized[:3])
print("="*50)

fullgram_X_test_text_tokenized = get_all_ngrams(X_test_text_tokenized)
print("full gram tokenization : \n", fullgram_X_test_text_tokenized[:3])
print("="*50)

Before Tokenization : 
 ['حساء خفافيش الصين ابداع القرف يعني', 'اجراءات هكذا استعدت مصر لمواجهه فيروس كورونا مرايتي', 'ايش الفايده لابس كمامات ومو لابس قفازات']
After Tokenization : 
 [['حساء', 'خفافيش', 'الصين', 'ابداع', 'القرف', 'يعني'], ['اجراءات', 'هكذا', 'استعدت', 'مصر', 'لمواجهه', 'فيروس', 'كورونا', 'مرايتي'], ['ايش', 'الفايده', 'لابس', 'كمامات', 'ومو', 'لابس', 'قفازات']]
full gram tokenization : 
 [['حساء', 'خفافيش', 'الصين', 'ابداع', 'القرف', 'يعني', 'حساء_خفافيش', 'خفافيش_الصين', 'الصين_ابداع', 'ابداع_القرف', 'القرف_يعني', 'حساء_خفافيش_الصين', 'خفافيش_الصين_ابداع', 'الصين_ابداع_القرف', 'ابداع_القرف_يعني'], ['اجراءات', 'هكذا', 'استعدت', 'مصر', 'لمواجهه', 'فيروس', 'كورونا', 'مرايتي', 'اجراءات_هكذا', 'هكذا_استعدت', 'استعدت_مصر', 'مصر_لمواجهه', 'لمواجهه_فيروس', 'فيروس_كورونا', 'كورونا_مرايتي', 'اجراءات_هكذا_استعدت', 'هكذا_استعدت_مصر', 'استعدت_مصر_لمواجهه', 'مصر_لمواجهه_فيروس', 'لمواجهه_فيروس_كورونا', 'فيروس_كورونا_مرايتي'], ['ايش', 'الفايده', 'لابس', 'كمامات', 'ومو', 'لابس', 'قفاز

In [14]:
rezk_model = keras_load_model("../ml_models_saved/dl_models/run_with_rezk_skipgram_NS_word2vec_Rmsprob_lstm_with_batch_learning_rate=5e-05__model.h5"  )

X_test_embed_matrix = text_to_matrix_using_word2vec(our_word2vec_model, fullgram_X_test_text_tokenized, max_len_str)
X_test_embed_matrix = X_test_embed_matrix.reshape(X_test_embed_matrix.shape[0], max_len_str, number_of_features)

keras_f1_score_result(rezk_model, X_test_embed_matrix, y_test)
print("="*50)

(180, 237, 300)
(180, 71100)
[-3.567e-04 -8.929e-02  2.818e-02  2.356e-01  1.252e-01  3.792e-03
 -2.191e-02  5.156e-01  1.274e-01 -2.681e-01 -3.408e-01 -1.803e-01
 -1.157e-01  1.272e-01 -3.137e-01 -2.996e-01  1.681e-01 -7.983e-02
 -1.969e-01 -5.366e-01 -3.218e-03 -3.503e-01  2.489e-01 -1.070e-01
  4.443e-01  1.738e-02 -2.444e-01 -5.503e-01 -3.960e-01  3.381e-01
  1.674e-01 -5.487e-02  3.347e-01  1.087e-01  2.939e-01  1.553e-01
  3.411e-01  2.720e-01  3.496e-03 -2.512e-01  2.664e-01  2.593e-01
  8.331e-02 -1.775e-01  2.263e-01  1.796e-01  1.604e-01 -5.878e-02
  2.223e-01  3.082e-02]
===================== Validate Result =====================
F1 score is:  0.9222222222222223
